In [94]:
# pip install opencv-python
# pip install ultralytics

In [95]:
import cv2
import os
from ultralytics import YOLO

In [96]:
model = YOLO('runs/detect/train/weights/best.pt')

In [97]:
def delete_files_in_folder(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            file_path = os.path.join(root, file_name)
            try:
                os.unlink(file_path)
            except Exception as e:
                print(f"Error deleting file: {file_path} -- {e}")

In [98]:
directory = '/Users/muzafarov/Desktop/MyEducation/DataEngineering-Course/Module 3. Active learning/new_image'
img_list = os.listdir(directory)
if '.DS_Store' in img_list:
    os.remove ('/Users/muzafarov/Desktop/MyEducation/DataEngineering-Course/Module 3. Active learning/new_image/.DS_Store')
print(img_list)


['8.png', '9.png', '11.png', '10.png', '4.png', '5.png', '7.png', '6.png', '2.png', '3.png', '1.png']


In [99]:
train_images_path = '/Users/muzafarov/Desktop/MyEducation/DataEngineering-Course/Module 3. Active learning/my_dataset_yolo/images/train'
print(f"В папке имеется {len(img_list)} изображений")

В папке имеется 11 изображений


In [100]:
delete_files_in_folder('/Users/muzafarov/Desktop/MyEducation/DataEngineering-Course/Module 3. Active learning/my_dataset_yolo/images/train')
delete_files_in_folder('/Users/muzafarov/Desktop/MyEducation/DataEngineering-Course/Module 3. Active learning/my_dataset_yolo/labels/train')

In [101]:
for img_name in img_list:
    img_filepath = directory +"/"+ img_name
    print(img_filepath)
    img = cv2.imread(img_filepath)
    img_copy = img
    
    # предобработка изображения как перед обучением
    # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # gray_three = cv2.merge([gray,gray,gray])
    # img = gray_three
    
    # получаем ширину и высоту картинки
    h, w, _ = img.shape
    
    # получаем предсказания по картинке
    results = model.predict(source=img, conf=0.50)
    
    # расшифровываем объект results
    bboxes_ = results[0].boxes.xyxy.tolist()
    bboxes = list(map(lambda x: list(map(lambda y: int(y), x)), bboxes_))
    confs_ = results[0].boxes.conf.tolist()
    confs = list(map(lambda x: int(x*100), confs_))
    classes_ = results[0].boxes.cls.tolist()
    classes = list(map(lambda x: int(x), classes_))
    cls_dict = results[0].names
    class_names = list(map(lambda x: cls_dict[x], classes))
    
    # приводим дешифрированные данные в удобный вид
    annot_lines = []
    for index, val in enumerate(class_names):
        xmin, ymin, xmax, ymax = int(bboxes[index][0]), int(bboxes[index][1]), int(bboxes[index][2]), int(bboxes[index][3])
        width = xmax - xmin
        height = ymax - ymin
        center_x = xmin + (width/2)
        center_y = ymin + (height/2) 
        annotation = f"{classes[index]} {center_x/w} {center_y/h} {width/w} {height/h}"
        annot_lines.append(annotation)
    
    # копируем картинку в папку базы изображений для импорта
    cv2.imwrite(os.path.join(train_images_path , img_name), img_copy)
    
    # записываем файл аннотации в папку базы изображений для импорта
    txt_name = img_name.replace(".png",".txt")
    with open(f'/Users/muzafarov/Desktop/MyEducation/DataEngineering-Course/Module 3. Active learning/my_dataset_yolo/labels/train/{txt_name}', 'w') as f:
        for line in annot_lines:
            f.write(line)
            f.write('\n')

            
        
    

/Users/muzafarov/Desktop/MyEducation/DataEngineering-Course/Module 3. Active learning/new_image/8.png

0: 416x640 1 bird, 358.2ms
Speed: 3.2ms preprocess, 358.2ms inference, 0.6ms postprocess per image at shape (1, 3, 416, 640)
/Users/muzafarov/Desktop/MyEducation/DataEngineering-Course/Module 3. Active learning/new_image/9.png

0: 416x640 2 birds, 191.5ms
Speed: 3.6ms preprocess, 191.5ms inference, 0.3ms postprocess per image at shape (1, 3, 416, 640)
/Users/muzafarov/Desktop/MyEducation/DataEngineering-Course/Module 3. Active learning/new_image/11.png

0: 416x640 3 birds, 182.3ms
Speed: 1.6ms preprocess, 182.3ms inference, 0.5ms postprocess per image at shape (1, 3, 416, 640)
/Users/muzafarov/Desktop/MyEducation/DataEngineering-Course/Module 3. Active learning/new_image/10.png

0: 416x640 2 birds, 171.2ms
Speed: 1.6ms preprocess, 171.2ms inference, 0.5ms postprocess per image at shape (1, 3, 416, 640)
/Users/muzafarov/Desktop/MyEducation/DataEngineering-Course/Module 3. Active learni